In [49]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
import umap.umap_ as umap
import random
from sklearn.feature_selection import SelectKBest, f_classif, RFE
from sklearn.pipeline import Pipeline
from scipy.stats import randint, uniform, loguniform
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression

In [50]:
data_dir = './GSE61260/'

In [51]:
df = pd.read_csv(os.path.join(data_dir, 'combined.csv'))

df.head()

,Sample_ID,ENSG00000000003,ENSG00000000005,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,...,ENSG00000284032,ENSG00000284373,ENSG00000284387,ENSG00000284395,ENSG00000284505,ENSG00000284552,Disease,characteristics_ch1_age,characteristics_ch1_bmi,characteristics_ch1_Sex_male
0,GSM1501013,8.002176,0.175238,4.949994,3.285351,0.959516,0.672894,59.325067,4.168056,8.104327,...,1.222211,0.132392,0.491394,-0.079424,-0.255411,3.071849,normal control,70,27.1,1
1,GSM1501014,23.292853,-0.031148,5.657237,2.786927,1.203578,0.358625,46.002714,4.889903,7.588060,...,0.487849,-0.066907,0.195198,0.178077,-0.190626,1.399320,healthy obese,49,30.5,0
2,GSM1501015,10.557890,0.097413,7.680354,2.795125,0.789783,0.733455,43.835288,5.133417,9.559194,...,0.423539,-0.155705,0.086307,0.230390,-0.489850,1.632879,normal control,76,25.3,0
3,GSM1501016,10.272135,0.066298,8.835539,3.166677,0.941032,0.755181,51.390227,4.168056,10.980774,...,0.357365,-0.321937,0.232894,-0.066718,-0.233382,2.117271,normal control,48,25.8,1
4,GSM1501017,8.290414,-0.039647,5.792685,2.203175,0.846344,0.612263,55.423440,3.657327,10.710821,...,0.452607,-0.263615,0.443599,0.291308,-0.474034,3.032463,normal control,73,23.5,0


In [52]:
le = LabelEncoder()

y = le.fit_transform(df['Disease'])

print(le.classes_)

X = df.drop(columns = ['Sample_ID', 'Disease'])

['healthy obese' 'nafld' 'nash' 'normal control' 'pbc' 'psc']


In [53]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

print(X_train.shape)
print(X_test.shape)

(106, 21664)
(27, 21664)


In [54]:
pipeline_steps = [
    ('scaler', StandardScaler()),
    ('rfe', RFE(estimator=LogisticRegression(random_state=42, solver='liblinear', max_iter=1000))),
    ('classifier', XGBClassifier(random_state=42, tree_method='hist', enable_categorical=False)) # Use 'hist' for speed
]
pipeline = Pipeline(pipeline_steps)

In [ ]:
max_features_for_rfe = min(X_train.shape[1], X_train.shape[0] - 1) 

parameters = {
    # RFE parameters:
    # 'rfe__n_features_to_select': The number of features to select (the target number of genes)
    'rfe__n_features_to_select': randint(50, max_features_for_rfe), # Try selecting between 50 and max_features
    # 'rfe__step': The number (or fraction) of features to remove at each iteration.
    # A smaller step leads to a more thorough but slower search.
    'rfe__step': randint(1, 10), # Remove 1 to 9 features at each step

    # XGBoost Classifier Parameters
    'classifier__n_estimators': randint(100, 1000), # Number of boosting rounds (trees)
    'classifier__learning_rate': loguniform(0.001, 0.3), # Step size shrinkage to prevent overfitting
    'classifier__max_depth': randint(3, 10), # Max depth of a tree. Typical range is 3-10
    'classifier__subsample': uniform(0.6, 0.4), # Subsample ratio of the training instance (per tree)
    'classifier__colsample_bytree': uniform(0.6, 0.4), # Subsample ratio of columns when constructing each tree
    'classifier__gamma': loguniform(1e-9, 1e-1), # Minimum loss reduction required to make a further partition
    'classifier__min_child_weight': randint(1, 10), # Minimum sum of instance weight (hessian) needed in a child
    'classifier__reg_alpha': loguniform(1e-9, 1e-1), # L1 regularization term on weights
    'classifier__reg_lambda': loguniform(1e-9, 1e-1), # L2 regularization term on weights
    'classifier__objective': ['multi:softmax'], # For multi-class classification
    'classifier__eval_metric': ['mlogloss'], # Metric used for validation set (for early stopping)
    'classifier__use_label_encoder': [False], # Suppress a future warning if needed
    'classifier__class_weight': [None, 'balanced'] # Handle imbalance if any
}

# Initialize RandomizedSearchCV
random_search_pipeline = RandomizedSearchCV(
    estimator=pipeline,                
    param_distributions=parameters,  
    n_iter=20,                        # Start with 100-200 iterations for a thorough search
    cv=5,                              # 5-fold cross-validation
    scoring='f1_weighted',             # Recommended for imbalanced classes; AUC-ROC also good
    verbose=1,                         
    random_state=42,                   
    n_jobs=-1                          
)

print("\nStarting RandomizedSearchCV on Pipeline...")
random_search_pipeline.fit(X_train, y_train)
print("RandomizedSearchCV on Pipeline complete.")

print("\nBest hyperparameters found for the pipeline:")
print(random_search_pipeline.best_params_)

print("\nBest cross-validation score:")
print(random_search_pipeline.best_score_)

best_pipeline_model = random_search_pipeline.best_estimator_
print("\nBest Pipeline Model:")
print(best_pipeline_model)

y_pred_tuned = best_pipeline_model.predict(X_test)

print("\n--- Evaluation of Best Tuned Pipeline on Test Set ---")
print(f"Accuracy: {accuracy_score(y_test, y_pred_tuned):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_tuned, target_names=le.classes_))



Starting RandomizedSearchCV on Pipeline...
Fitting 5 folds for each of 100 candidates, totalling 500 fits


/Users/jaydn/Documents/GitHub/Human-Liver-RNA-Seq-Gene-Expression-Analysis/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another solver which supports the multinomial loss or wrap the estimator in a OneVsRestClassifier to keep applying a one-versus-rest scheme.
  warnings.warn(
/Users/jaydn/Documents/GitHub/Human-Liver-RNA-Seq-Gene-Expression-Analysis/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another solver which supports the multinomial loss or wrap the estimator in a OneVsRestClassifier to keep applying a one-versus-rest scheme.
  warnings.warn(
/Users/jaydn/Documents/GitHub/Human-Liver-RNA-Seq-Gene-Expression-Analysis/venv/lib/python3.12/site-packages/sklearn/linea